In [30]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


In [31]:
df = pd.read_csv('../../data/curated/individual_property_final.csv')

In [32]:
# Select relevant features and target (assuming 'Cost' is the target)
df = df.drop(columns=['Address', 'Latitude', 'Longitude', 'Postcode', 'SA2_CODE21', 'LGA_CODE24', 'Suburb'], axis=1)

categorical_columns = ['Property Type','Closest Gov Secondary School']
# Apply LabelEncoder to each categorical column
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le
    
df_train = df[(df['Year'] >= 2015) & (df['Year'] <= 2024)]
df_predict = df[(df['Year'] >= 2025) & (df['Year'] <= 2027)]

features = df_train.drop(columns=['Cost'])
target = df_train['Cost']

# Normalize the features and target using MinMaxScaler (LSTMs work better with normalized data)
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)
target_scaled = scaler.fit_transform(target.values.reshape(-1, 1))

# Convert data into sequences for LSTM
def create_sequences(features, target, time_steps=10):
    X, y = [], []
    for i in range(len(features) - time_steps):
        X.append(features[i:i+time_steps])
        y.append(target[i+time_steps])
    return np.array(X), np.array(y)

# Set the number of time steps (e.g., 10 previous timesteps to predict the next one)
time_steps = 10
X, y = create_sequences(features_scaled, target_scaled, time_steps)

# Display the shape of the data
print(f'X shape: {X.shape}, y shape: {y.shape}')


X shape: (62110, 10, 21), y shape: (62110, 1)


In [33]:

# Build a simple LSTM model
model = Sequential()

# Add the LSTM layer with 50 units
model.add(LSTM(units=50, return_sequences=False, input_shape=(X.shape[1], X.shape[2])))

# Optionally, add Dropout for regularization
model.add(Dropout(0.2))

# Add a Dense layer for the output
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Display model summary
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 50)                14400     
                                                                 
 dropout_3 (Dropout)         (None, 50)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 14451 (56.45 KB)
Trainable params: 14451 (56.45 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
seed=37

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50
1553/1553 [==============================] - 4s 2ms/step - loss: 0.0109 - val_loss: 0.0084
Epoch 2/50
1553/1553 [==============================] - 3s 2ms/step - loss: 0.0090 - val_loss: 0.0081
Epoch 3/50
1553/1553 [==============================] - 3s 2ms/step - loss: 0.0086 - val_loss: 0.0080
Epoch 4/50
1553/1553 [==============================] - 3s 2ms/step - loss: 0.0077 - val_loss: 0.0072
Epoch 5/50
1553/1553 [==============================] - 3s 2ms/step - loss: 0.0073 - val_loss: 0.0069
Epoch 6/50
1553/1553 [==============================] - 3s 2ms/step - loss: 0.0072 - val_loss: 0.0068
Epoch 7/50
1553/1553 [==============================] - 3s 2ms/step - loss: 0.0071 - val_loss: 0.0066
Epoch 8/50
1553/1553 [==============================] - 3s 2ms/step - loss: 0.0070 - val_loss: 0.0066
Epoch 9/50
1553/1553 [==============================] - 3s 2ms/step - loss: 0.0069 - val_loss: 0.0066
Epoch 10/50
1553/1553 [==============================] - 3s 2ms/step - loss: 0.006

In [35]:
# Evaluate the model on the test set
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

# Make predictions
y_pred = model.predict(X_test)

# Optionally, inverse transform the scaled predictions and true values
y_pred_inverse = scaler.inverse_transform(y_pred)
y_test_inverse = scaler.inverse_transform(y_test.reshape(-1, 1))

# Display the predictions and true values
print(f'Predicted: {y_pred_inverse.flatten()}')
print(f'Actual: {y_test_inverse.flatten()}')

389/389 [==============================] - 0s 822us/step - loss: 0.0042
Test Loss: 0.00424437178298831
389/389 [==============================] - 1s 861us/step
Predicted: [323.5034  241.95937 437.60992 ... 188.8427  186.12582 395.8115 ]
Actual: [316.2962963  230.41666667 470.         ... 164.25352113 158.12154696
 449.93697479]


In [36]:

# Make predictions on the test set
y_pred = model.predict(X_test)

# Inverse transform the scaled predictions and true values (if you scaled the data)
y_pred_inverse = scaler.inverse_transform(y_pred)
y_test_inverse = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test_inverse, y_pred_inverse))
print(f'RMSE: {rmse}')


389/389 [==============================] - 0s 781us/step
RMSE: 88.13204808745958


Looks bad, skip